In [1]:
!pip install pyspark
!pip install folium
!pip install geopy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 7.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=b80f9bddbb323c48273a9bb63e59bcb493de22680a83ccb62f8c03d2a46ca2a8
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from pyspark.sql import types, Window, functions as F, SparkSession


In [76]:
# Get hurricane data

#hurricane_data = pd.read_csv('/content/ibtracs.EP.list.csv', nrows=500)
hurricane_data = pd.read_csv('/content/ibtracs.NA.csv')

hurricane_df = pd.DataFrame(hurricane_data, columns=['SID', 'SEASON', 'BASIN', 'LAT', 'LON', 'USA_ROCI', 'USA_RMW', 'ISO_TIME', 'BOM_EYE', 'NATURE', 'USA_STATUS'])
hurricane_df = hurricane_df[(hurricane_df["NATURE"] == 'TS') & ((hurricane_df["USA_STATUS"] == 'HU') | (hurricane_df["USA_STATUS"] == 'HR'))]
hurricane_df.drop(hurricane_df.index[0], inplace=True)
hurricane_df["ISO_TIME"] = pd.to_datetime(hurricane_df["ISO_TIME"], errors='coerce')
hurricane_df["MONTH"] = hurricane_df["ISO_TIME"].dt.month
hurricane_df['USA_RMW'] = pd.to_numeric(hurricane_df['USA_RMW'], errors='coerce')
hurricane_df['USA_ROCI'] = pd.to_numeric(hurricane_df['USA_ROCI'], errors='coerce')

hurricane_df = hurricane_df.dropna(subset=['ISO_TIME', 'USA_RMW', 'USA_ROCI'])

hurricane_df['RADIUS_KM'] = hurricane_df['USA_RMW'].apply(lambda x: float(x) * 1.852)
hurricane_df['LON'] = pd.to_numeric(hurricane_df['LON'], errors='coerce')
hurricane_df['LAT'] = pd.to_numeric(hurricane_df['LAT'], errors='coerce')

hurricane_df.dropna(subset=['LAT', 'LON'], inplace=True)

hurricane_df['Datetime'] = pd.to_datetime(hurricane_df['ISO_TIME'])

hurricane_df['YEAR'] = hurricane_df['Datetime'].dt.year
hurricane_df['MONTH'] = hurricane_df['Datetime'].dt.month
hurricane_df['DAY'] = hurricane_df['Datetime'].dt.day


hurricane_df


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,8,9,14,19,20,23,24,161,162) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-76-e4605229f88c>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricane_df['RADIUS_KM'] = hurricane_df['USA_RMW'].apply(lambda x: float(x) * 1.852)
<ipython-input-76-e4605229f88c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hu

,SID,SEASON,BASIN,LAT,LON,USA_ROCI,USA_RMW,ISO_TIME,BOM_EYE,NATURE,USA_STATUS,MONTH,RADIUS_KM,Datetime,YEAR,DAY
103074,2001245N13326,2001,NaN,29.7000,-60.4000,120.0,35.0,2001-09-09 00:00:00,,TS,HU,9,64.820,2001-09-09 00:00:00,2001,9
103082,2001245N13326,2001,NaN,33.3000,-63.3000,120.0,30.0,2001-09-10 00:00:00,,TS,HU,9,55.560,2001-09-10 00:00:00,2001,10
103088,2001245N13326,2001,NaN,35.7000,-65.4000,120.0,20.0,2001-09-10 18:00:00,,TS,HU,9,37.040,2001-09-10 18:00:00,2001,10
103092,2001245N13326,2001,NaN,36.9000,-65.9000,120.0,20.0,2001-09-11 06:00:00,,TS,HU,9,37.040,2001-09-11 06:00:00,2001,11
103093,2001245N13326,2001,NaN,37.1575,-65.8027,150.0,20.0,2001-09-11 09:00:00,,TS,HU,9,37.040,2001-09-11 09:00:00,2001,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125590,2022304N34298,2022,NaN,46.0971,-36.8003,210.0,47.0,2022-11-03 15:00:00,,TS,HU,11,87.044,2022-11-03 15:00:00,2022,3
125630,2022309N16290,2022,NaN,26.7000,-78.4000,280.0,15.0,2022-11-10 00:00:00,,TS,HU,11,27.780,2022-11-10 00:00:00,2022,10
125631,2022309N16290,2022,NaN,26.9623,-79.1050,280.0,15.0,2022-11-10 03:00:00,,TS,HU,11,27.780,2022-11-10 03:00:00,2022,10
125632,2022309N16290,2022,NaN,27.3000,-79.9000,280.0,15.0,2022-11-10 06:00:00,,TS,HU,11,27.780,2022-11-10 06:00:00,2022,10


In [77]:
# Get only 1 month data 09/2022 for testing
filtered_hurricane_df_1_month = hurricane_df[(hurricane_df["SEASON"] == 2022) & (hurricane_df["MONTH"] == 9)]
filtered_hurricane_df_1_month = filtered_hurricane_df_1_month.reset_index(drop=True)
filtered_hurricane_df_1_month


,SID,SEASON,BASIN,LAT,LON,USA_ROCI,USA_RMW,ISO_TIME,BOM_EYE,NATURE,USA_STATUS,MONTH,RADIUS_KM,Datetime,YEAR,DAY
0,2022239N12324,2022,NaN,24.6000,-65.8000,180.0,30.0,2022-09-07 00:00:00,,TS,HU,9,55.56,2022-09-07 00:00:00,2022,7
1,2022239N12324,2022,NaN,24.8225,-65.8650,200.0,30.0,2022-09-07 03:00:00,,TS,HU,9,55.56,2022-09-07 03:00:00,2022,7
2,2022239N12324,2022,NaN,25.0000,-65.9000,220.0,30.0,2022-09-07 06:00:00,,TS,HU,9,55.56,2022-09-07 06:00:00,2022,7
3,2022239N12324,2022,NaN,25.1775,-65.8725,220.0,30.0,2022-09-07 09:00:00,,TS,HU,9,55.56,2022-09-07 09:00:00,2022,7
4,2022239N12324,2022,NaN,25.4000,-65.8000,220.0,30.0,2022-09-07 12:00:00,,TS,HU,9,55.56,2022-09-07 12:00:00,2022,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,2022263N10313,2022,NaN,30.7742,-79.0129,195.0,40.0,2022-09-30 09:00:00,,TS,HU,9,74.08,2022-09-30 09:00:00,2022,30
143,2022263N10313,2022,NaN,31.5000,-79.0000,210.0,40.0,2022-09-30 12:00:00,,TS,HU,9,74.08,2022-09-30 12:00:00,2022,30
144,2022263N10313,2022,NaN,32.3975,-79.0794,210.0,40.0,2022-09-30 15:00:00,,TS,HU,9,74.08,2022-09-30 15:00:00,2022,30
145,2022263N10313,2022,NaN,33.3000,-79.2000,210.0,40.0,2022-09-30 18:00:00,,TS,HU,9,74.08,2022-09-30 18:00:00,2022,30


In [78]:
# Get 1 year hurricane data 2022 for testing
filtered_hurricane_df_2022 = hurricane_df[(hurricane_df["SEASON"] == 2022)]
filtered_hurricane_df_2022 = filtered_hurricane_df_2022.reset_index(drop=True)
filtered_hurricane_df_2022

,SID,SEASON,BASIN,LAT,LON,USA_ROCI,USA_RMW,ISO_TIME,BOM_EYE,NATURE,USA_STATUS,MONTH,RADIUS_KM,Datetime,YEAR,DAY
0,2022174N06341,2022,EP,12.9000,-94.7000,90.0,15.0,2022-07-04 00:00:00,,TS,HU,7,27.780,2022-07-04 00:00:00,2022,4
1,2022174N06341,2022,EP,13.1875,-95.5050,90.0,15.0,2022-07-04 03:00:00,,TS,HU,7,27.780,2022-07-04 03:00:00,2022,4
2,2022174N06341,2022,EP,13.4000,-96.4000,90.0,15.0,2022-07-04 06:00:00,,TS,HU,7,27.780,2022-07-04 06:00:00,2022,4
3,2022174N06341,2022,EP,13.5301,-97.3302,90.0,15.0,2022-07-04 09:00:00,,TS,HU,7,27.780,2022-07-04 09:00:00,2022,4
4,2022174N06341,2022,EP,13.6000,-98.2000,90.0,15.0,2022-07-04 12:00:00,,TS,HU,7,27.780,2022-07-04 12:00:00,2022,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2022304N34298,2022,NaN,46.0971,-36.8003,210.0,47.0,2022-11-03 15:00:00,,TS,HU,11,87.044,2022-11-03 15:00:00,2022,3
205,2022309N16290,2022,NaN,26.7000,-78.4000,280.0,15.0,2022-11-10 00:00:00,,TS,HU,11,27.780,2022-11-10 00:00:00,2022,10
206,2022309N16290,2022,NaN,26.9623,-79.1050,280.0,15.0,2022-11-10 03:00:00,,TS,HU,11,27.780,2022-11-10 03:00:00,2022,10
207,2022309N16290,2022,NaN,27.3000,-79.9000,280.0,15.0,2022-11-10 06:00:00,,TS,HU,11,27.780,2022-11-10 06:00:00,2022,10


In [79]:
# Get 2 year hurricane data 2021 & 2022 for testing
filtered_hurricane_df_2021_2022 = hurricane_df[(hurricane_df["SEASON"] == 2022) | (hurricane_df["SEASON"] == 2021)]
filtered_hurricane_df_2021_2022 = filtered_hurricane_df_2021_2022.reset_index(drop=True)
filtered_hurricane_df_2021_2022

,SID,SEASON,BASIN,LAT,LON,USA_ROCI,USA_RMW,ISO_TIME,BOM_EYE,NATURE,USA_STATUS,MONTH,RADIUS_KM,Datetime,YEAR,DAY
0,2021182N09317,2021,NaN,13.0000,-59.8000,140.0,20.0,2021-07-02 12:00:00,,TS,HU,7,37.040,2021-07-02 12:00:00,2021,2
1,2021182N09317,2021,NaN,13.3843,-61.0855,140.0,17.0,2021-07-02 15:00:00,,TS,HU,7,31.484,2021-07-02 15:00:00,2021,2
2,2021182N09317,2021,NaN,13.8000,-62.4000,140.0,15.0,2021-07-02 18:00:00,,TS,HU,7,27.780,2021-07-02 18:00:00,2021,2
3,2021182N09317,2021,NaN,14.2342,-63.7356,140.0,15.0,2021-07-02 21:00:00,,TS,HU,7,27.780,2021-07-02 21:00:00,2021,2
4,2021182N09317,2021,NaN,14.7000,-65.1000,140.0,15.0,2021-07-03 00:00:00,,TS,HU,7,27.780,2021-07-03 00:00:00,2021,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,2022304N34298,2022,NaN,46.0971,-36.8003,210.0,47.0,2022-11-03 15:00:00,,TS,HU,11,87.044,2022-11-03 15:00:00,2022,3
433,2022309N16290,2022,NaN,26.7000,-78.4000,280.0,15.0,2022-11-10 00:00:00,,TS,HU,11,27.780,2022-11-10 00:00:00,2022,10
434,2022309N16290,2022,NaN,26.9623,-79.1050,280.0,15.0,2022-11-10 03:00:00,,TS,HU,11,27.780,2022-11-10 03:00:00,2022,10
435,2022309N16290,2022,NaN,27.3000,-79.9000,280.0,15.0,2022-11-10 06:00:00,,TS,HU,11,27.780,2022-11-10 06:00:00,2022,10


In [80]:
# Draw hurricanes on the map
# import folium

mean_lat = filtered_hurricane_df_2021_2022['LAT'].mean()
mean_lon = filtered_hurricane_df_2021_2022['LON'].mean()
map_hurricanes = folium.Map(location=[mean_lat, mean_lon], zoom_start=5, min_zoom=4, max_zoom=7)

# #FFA500
# mediumpurple
for i, row in filtered_hurricane_df_2021_2022.iterrows():
    lat = row['LAT']
    lon = row['LON']
    folium.CircleMarker(location=[lat, lon], 
                        radius_fixed= True,
                        radius = row['RADIUS_KM'], 
                        fill=True, color='mediumpurple', opacity = 0.3, fill_color = 'mediumpurple', fill_opacity=0.3, 
                        popup = row['ISO_TIME'],
                        tooltip='LAT: {} \LON: {} \RADIUS_KM: {}'.format(row['LAT'],row['LON'],row['RADIUS_KM'])).add_to(map_hurricanes)

map_hurricanes

In [81]:
# Get the zillow data (2000 - 2022)

zillow_data = pd.read_csv('/content/zillow_city.csv')
#loan_data = pd.read_csv('/content/loans.csv', nrows = 1000)

zillow_df = pd.DataFrame(zillow_data)
states = ['NC', 'SC', 'GA', 'LA', 'FL']
#south_eastern_states = ["AL", "FL", "GA", "MS", "SC", "TN", "KY", "NC", "VA", "WV", "DC"]
zillow_df = zillow_df[zillow_df['State'].isin(states)]
zillow_df = zillow_df.rename(columns={'RegionName': 'CityName'})

zillow_df


,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,2000-01-31,2000-02-29,...,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31
12,25290,12,Jacksonville,city,FL,FL,"Jacksonville, FL",Duval County,102258.0,102568.0,...,287281.0,294526.0,300924.0,306224.0,309823.0,312608.0,313544.0,313596.0,313426.0,312639.0
13,24043,13,Charlotte,city,NC,NC,"Charlotte-Concord-Gastonia, NC-SC",Mecklenburg County,156927.0,157185.0,...,370686.0,380659.0,389298.0,394810.0,397133.0,397961.0,398041.0,397126.0,397207.0,397154.0
20,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,114429.0,114802.0,...,347693.0,358122.0,368304.0,376668.0,381557.0,384365.0,386208.0,387141.0,387136.0,385910.0
29,12700,29,Miami,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Miami-Dade County,145831.0,146611.0,...,483182.0,492711.0,507481.0,523059.0,539525.0,549940.0,555524.0,558137.0,561458.0,562992.0
40,54047,40,Raleigh,city,NC,NC,"Raleigh-Cary, NC",Wake County,176360.0,176539.0,...,429010.0,440469.0,451518.0,459246.0,459246.0,454888.0,449661.0,447103.0,441792.0,435554.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22451,6461,28439,Penney Farms,city,FL,FL,"Jacksonville, FL",Clay County,NaN,NaN,...,244465.0,250477.0,255027.0,257984.0,261891.0,265824.0,269024.0,269453.0,268313.0,267397.0
22457,49256,28439,Dillsboro,city,NC,NC,"Cullowhee, NC",Jackson County,NaN,NaN,...,295121.0,290556.0,291611.0,296391.0,301924.0,307213.0,309247.0,310866.0,310327.0,308937.0
22476,26569,28439,Polk City,city,FL,FL,"Lakeland-Winter Haven, FL",Polk County,91602.0,88502.0,...,229759.0,236189.0,242029.0,245667.0,249104.0,251393.0,252815.0,251973.0,249916.0,249046.0
22479,42861,28439,Oak Park,city,GA,GA,NaN,Emanuel County,NaN,NaN,...,63588.0,64611.0,65470.0,66885.0,68486.0,69674.0,70083.0,70215.0,69612.0,68641.0


In [82]:
# Import the city data to get lat and lon for properties
# This data can be changed later

city_data = pd.read_csv('/content/uscities.csv')
city_df = pd.DataFrame(city_data, columns=['city', 'state_id', 'lat', 'lng', 'zips'])
city_df

,city,state_id,lat,lng,zips
0,New York,NY,40.6943,-73.9249,11229 11228 11226 11225 11224 11222 11221 1122...
1,Los Angeles,CA,34.1141,-118.4068,91367 90291 90293 90292 91316 91311 90035 9003...
2,Chicago,IL,41.8375,-87.6866,60018 60649 60641 60640 60643 60642 60645 6064...
3,Miami,FL,25.7840,-80.2101,33128 33129 33125 33126 33127 33149 33144 3314...
4,Dallas,TX,32.7935,-96.7667,75098 75287 75230 75231 75236 75237 75235 7525...
...,...,...,...,...,...
30404,Drummond,ID,43.9996,-111.3433,83420
30405,Lost Springs,WY,42.7652,-104.9255,82224 82229
30406,Provo,SD,43.1937,-103.8329,57735
30407,Goldcreek,MT,46.5838,-112.9284,59733


In [83]:
# Get lat and long for each city
# Enrich zillow dataset

zillow_with_coords_df = pd.merge(zillow_df, city_df, left_on=['CityName', 'State'], right_on=['city', 'state_id'], how='left')
zillow_with_coords_df = zillow_with_coords_df.dropna(subset=['lat', 'lng'], how='any')
zillow_with_coords_df['lng'] = pd.to_numeric(zillow_with_coords_df['lng'], errors='coerce')
zillow_with_coords_df['lat'] = pd.to_numeric(zillow_with_coords_df['lat'], errors='coerce')
zillow_with_coords_df = zillow_with_coords_df.rename(columns={'lat': 'LAT', 'lng': 'LON'})
zillow_with_coords_df = zillow_with_coords_df.reset_index(drop=True)
zillow_with_coords_df


,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,2000-01-31,2000-02-29,...,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,city,state_id,LAT,LON,zips
0,25290,12,Jacksonville,city,FL,FL,"Jacksonville, FL",Duval County,102258.0,102568.0,...,312608.0,313544.0,313596.0,313426.0,312639.0,Jacksonville,FL,30.3322,-81.6749,32224 32225 32226 32227 32220 32221 32222 3222...
1,24043,13,Charlotte,city,NC,NC,"Charlotte-Concord-Gastonia, NC-SC",Mecklenburg County,156927.0,157185.0,...,397961.0,398041.0,397126.0,397207.0,397154.0,Charlotte,NC,35.2083,-80.8303,28269 28262 28105 28280 28282 28209 28208 2820...
2,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,114429.0,114802.0,...,384365.0,386208.0,387141.0,387136.0,385910.0,Orlando,FL,28.4773,-81.3370,32829 32827 32824 32822 32804 32805 32806 3280...
3,12700,29,Miami,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Miami-Dade County,145831.0,146611.0,...,549940.0,555524.0,558137.0,561458.0,562992.0,Miami,FL,25.7840,-80.2101,33128 33129 33125 33126 33127 33149 33144 3314...
4,54047,40,Raleigh,city,NC,NC,"Raleigh-Cary, NC",Wake County,176360.0,176539.0,...,454888.0,449661.0,447103.0,441792.0,435554.0,Raleigh,NC,35.8324,-78.6429,27617 27616 27615 27614 27613 27612 27610 2760...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2189,6461,28439,Penney Farms,city,FL,FL,"Jacksonville, FL",Clay County,NaN,NaN,...,265824.0,269024.0,269453.0,268313.0,267397.0,Penney Farms,FL,29.9807,-81.8108,32079 32043
2190,49256,28439,Dillsboro,city,NC,NC,"Cullowhee, NC",Jackson County,NaN,NaN,...,307213.0,309247.0,310866.0,310327.0,308937.0,Dillsboro,NC,35.3706,-83.2527,28779 28725
2191,26569,28439,Polk City,city,FL,FL,"Lakeland-Winter Haven, FL",Polk County,91602.0,88502.0,...,251393.0,252815.0,251973.0,249916.0,249046.0,Polk City,FL,28.1805,-81.8296,33868
2192,42861,28439,Oak Park,city,GA,GA,NaN,Emanuel County,NaN,NaN,...,69674.0,70083.0,70215.0,69612.0,68641.0,Oak Park,GA,32.3822,-82.3153,30471 30436


In [84]:
zillow_with_coords_df_copy = zillow_with_coords_df.drop(['state_id', 'city', 'zips'], axis=1)
zillow_with_coords_df_copy

,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,2000-01-31,2000-02-29,...,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,LAT,LON
0,25290,12,Jacksonville,city,FL,FL,"Jacksonville, FL",Duval County,102258.0,102568.0,...,300924.0,306224.0,309823.0,312608.0,313544.0,313596.0,313426.0,312639.0,30.3322,-81.6749
1,24043,13,Charlotte,city,NC,NC,"Charlotte-Concord-Gastonia, NC-SC",Mecklenburg County,156927.0,157185.0,...,389298.0,394810.0,397133.0,397961.0,398041.0,397126.0,397207.0,397154.0,35.2083,-80.8303
2,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,114429.0,114802.0,...,368304.0,376668.0,381557.0,384365.0,386208.0,387141.0,387136.0,385910.0,28.4773,-81.3370
3,12700,29,Miami,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Miami-Dade County,145831.0,146611.0,...,507481.0,523059.0,539525.0,549940.0,555524.0,558137.0,561458.0,562992.0,25.7840,-80.2101
4,54047,40,Raleigh,city,NC,NC,"Raleigh-Cary, NC",Wake County,176360.0,176539.0,...,451518.0,459246.0,459246.0,454888.0,449661.0,447103.0,441792.0,435554.0,35.8324,-78.6429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2189,6461,28439,Penney Farms,city,FL,FL,"Jacksonville, FL",Clay County,NaN,NaN,...,255027.0,257984.0,261891.0,265824.0,269024.0,269453.0,268313.0,267397.0,29.9807,-81.8108
2190,49256,28439,Dillsboro,city,NC,NC,"Cullowhee, NC",Jackson County,NaN,NaN,...,291611.0,296391.0,301924.0,307213.0,309247.0,310866.0,310327.0,308937.0,35.3706,-83.2527
2191,26569,28439,Polk City,city,FL,FL,"Lakeland-Winter Haven, FL",Polk County,91602.0,88502.0,...,242029.0,245667.0,249104.0,251393.0,252815.0,251973.0,249916.0,249046.0,28.1805,-81.8296
2192,42861,28439,Oak Park,city,GA,GA,NaN,Emanuel County,NaN,NaN,...,65470.0,66885.0,68486.0,69674.0,70083.0,70215.0,69612.0,68641.0,32.3822,-82.3153


In [86]:
dates_string = "2000-01-31	2000-02-29	2000-03-31	2000-04-30	2000-05-31	2000-06-30	2000-07-31	2000-08-31	2000-09-30	2000-10-31	2000-11-30	2000-12-31	2001-01-31	2001-02-28	2001-03-31	2001-04-30	2001-05-31	2001-06-30	2001-07-31	2001-08-31	2001-09-30	2001-10-31	2001-11-30	2001-12-31	2002-01-31	2002-02-28	2002-03-31	2002-04-30	2002-05-31	2002-06-30	2002-07-31	2002-08-31	2002-09-30	2002-10-31	2002-11-30	2002-12-31	2003-01-31	2003-02-28	2003-03-31	2003-04-30	2003-05-31	2003-06-30	2003-07-31	2003-08-31	2003-09-30	2003-10-31	2003-11-30	2003-12-31	2004-01-31	2004-02-29	2004-03-31	2004-04-30	2004-05-31	2004-06-30	2004-07-31	2004-08-31	2004-09-30	2004-10-31	2004-11-30	2004-12-31	2005-01-31	2005-02-28	2005-03-31	2005-04-30	2005-05-31	2005-06-30	2005-07-31	2005-08-31	2005-09-30	2005-10-31	2005-11-30	2005-12-31	2006-01-31	2006-02-28	2006-03-31	2006-04-30	2006-05-31	2006-06-30	2006-07-31	2006-08-31	2006-09-30	2006-10-31	2006-11-30	2006-12-31	2007-01-31	2007-02-28	2007-03-31	2007-04-30	2007-05-31	2007-06-30	2007-07-31	2007-08-31	2007-09-30	2007-10-31	2007-11-30	2007-12-31	2008-01-31	2008-02-29	2008-03-31	2008-04-30	2008-05-31	2008-06-30	2008-07-31	2008-08-31	2008-09-30	2008-10-31	2008-11-30	2008-12-31	2009-01-31	2009-02-28	2009-03-31	2009-04-30	2009-05-31	2009-06-30	2009-07-31	2009-08-31	2009-09-30	2009-10-31	2009-11-30	2009-12-31	2010-01-31	2010-02-28	2010-03-31	2010-04-30	2010-05-31	2010-06-30	2010-07-31	2010-08-31	2010-09-30	2010-10-31	2010-11-30	2010-12-31	2011-01-31	2011-02-28	2011-03-31	2011-04-30	2011-05-31	2011-06-30	2011-07-31	2011-08-31	2011-09-30	2011-10-31	2011-11-30	2011-12-31	2012-01-31	2012-02-29	2012-03-31	2012-04-30	2012-05-31	2012-06-30	2012-07-31	2012-08-31	2012-09-30	2012-10-31	2012-11-30	2012-12-31	2013-01-31	2013-02-28	2013-03-31	2013-04-30	2013-05-31	2013-06-30	2013-07-31	2013-08-31	2013-09-30	2013-10-31	2013-11-30	2013-12-31	2014-01-31	2014-02-28	2014-03-31	2014-04-30	2014-05-31	2014-06-30	2014-07-31	2014-08-31	2014-09-30	2014-10-31	2014-11-30	2014-12-31	2015-01-31	2015-02-28	2015-03-31	2015-04-30	2015-05-31	2015-06-30	2015-07-31	2015-08-31	2015-09-30	2015-10-31	2015-11-30	2015-12-31	2016-01-31	2016-02-29	2016-03-31	2016-04-30	2016-05-31	2016-06-30	2016-07-31	2016-08-31	2016-09-30	2016-10-31	2016-11-30	2016-12-31	2017-01-31	2017-02-28	2017-03-31	2017-04-30	2017-05-31	2017-06-30	2017-07-31	2017-08-31	2017-09-30	2017-10-31	2017-11-30	2017-12-31	2018-01-31	2018-02-28	2018-03-31	2018-04-30	2018-05-31	2018-06-30	2018-07-31	2018-08-31	2018-09-30	2018-10-31	2018-11-30	2018-12-31	2019-01-31	2019-02-28	2019-03-31	2019-04-30	2019-05-31	2019-06-30	2019-07-31	2019-08-31	2019-09-30	2019-10-31	2019-11-30	2019-12-31	2020-01-31	2020-02-29	2020-03-31	2020-04-30	2020-05-31	2020-06-30	2020-07-31	2020-08-31	2020-09-30	2020-10-31	2020-11-30	2020-12-31	2021-01-31	2021-02-28	2021-03-31	2021-04-30	2021-05-31	2021-06-30	2021-07-31	2021-08-31	2021-09-30	2021-10-31	2021-11-30	2021-12-31	2022-01-31	2022-02-28	2022-03-31	2022-04-30	2022-05-31	2022-06-30	2022-07-31	2022-08-31	2022-09-30	2022-10-31	2022-11-30	2022-12-31"
dates_list = dates_string.split("\t")
print(dates_list)

['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31', '2000-11-30', '2000-12-31', '2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31', '2001-06-30', '2001-07-31', '2001-08-31', '2001-09-30', '2001-10-31', '2001-11-30', '2001-12-31', '2002-01-31', '2002-02-28', '2002-03-31', '2002-04-30', '2002-05-31', '2002-06-30', '2002-07-31', '2002-08-31', '2002-09-30', '2002-10-31', '2002-11-30', '2002-12-31', '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30', '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31', '2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31', '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-05-31', '2004-06-30', '2004-07-31', '2004-08-31', '2004-09-30', '2004-10-31', '2004-11-30', '2004-12-31', '2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-05-31', '2005-06-30', '2005-07-31', '2005-08-31', '2005-09-30', '2005-10-31', '2005-11-30', '2005

In [87]:
zillow_data_with_date = pd.melt(zillow_with_coords_df_copy, id_vars=['RegionID', 'SizeRank', 'CityName', 'RegionType', 'StateName', 'State', 'Metro', 'CountyName', 'LAT', 'LON'], value_vars=dates_list, var_name='Date', value_name='HomeValue')

zillow_data_with_date['Datetime'] = pd.to_datetime(zillow_data_with_date['Date'])

zillow_data_with_date['YEAR'] = zillow_data_with_date['Datetime'].dt.year
zillow_data_with_date['MONTH'] = zillow_data_with_date['Datetime'].dt.month
zillow_data_with_date['DAY'] = zillow_data_with_date['Datetime'].dt.day
zillow_data_with_date = zillow_data_with_date.dropna(subset=["HomeValue", "LAT", "LON"], how='any')
zillow_data_with_date

,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,LAT,LON,Date,HomeValue,Datetime,YEAR,MONTH,DAY
0,25290,12,Jacksonville,city,FL,FL,"Jacksonville, FL",Duval County,30.3322,-81.6749,2000-01-31,102258.0,2000-01-31,2000,1,31
1,24043,13,Charlotte,city,NC,NC,"Charlotte-Concord-Gastonia, NC-SC",Mecklenburg County,35.2083,-80.8303,2000-01-31,156927.0,2000-01-31,2000,1,31
2,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.3370,2000-01-31,114429.0,2000-01-31,2000,1,31
3,12700,29,Miami,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Miami-Dade County,25.7840,-80.2101,2000-01-31,145831.0,2000-01-31,2000,1,31
4,54047,40,Raleigh,city,NC,NC,"Raleigh-Cary, NC",Wake County,35.8324,-78.6429,2000-01-31,176360.0,2000-01-31,2000,1,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605539,6461,28439,Penney Farms,city,FL,FL,"Jacksonville, FL",Clay County,29.9807,-81.8108,2022-12-31,267397.0,2022-12-31,2022,12,31
605540,49256,28439,Dillsboro,city,NC,NC,"Cullowhee, NC",Jackson County,35.3706,-83.2527,2022-12-31,308937.0,2022-12-31,2022,12,31
605541,26569,28439,Polk City,city,FL,FL,"Lakeland-Winter Haven, FL",Polk County,28.1805,-81.8296,2022-12-31,249046.0,2022-12-31,2022,12,31
605542,42861,28439,Oak Park,city,GA,GA,NaN,Emanuel County,32.3822,-82.3153,2022-12-31,68641.0,2022-12-31,2022,12,31


In [91]:
# Version 3
def is_exposed_3(property_lat, property_long, hurricane_lat, hurricane_long, radius_km):
    #print("property lat: ", property_lat, " long: ", property_long)
    #print("hurricane lat: ", hurricane_lat, " long: ", hurricane_long, " radius_km: ", radius_km)
    property_point = (property_lat, property_long)
    hurricane_point = (hurricane_lat, hurricane_long)
    distance = geodesic(property_point, hurricane_point).km
    #print("distance: ", distance)
    if distance <= radius_km:
        return 2 # Exposed
    elif distance - radius_km < 20:
        return 1 # Risk, <20km to hurricane
    else:
        return 0 # Safe

In [92]:
# Version 3
# 1 month hurricane data + filtered states -> Took 10 minutes to run 
def update_exposed_status(zillow_data, hurricane_df):
    zillow_data_copy = zillow_data
    zillow_data_copy.reset_index(drop=True, inplace=True)
    zillow_data_copy['Exposed'] = 0

    for i in range(zillow_data_copy.shape[0]):
        current_status = zillow_data_copy.loc[i, 'Exposed']
        current_year = zillow_data_copy.loc[i, 'YEAR']
        current_month = zillow_data_copy.loc[i, 'MONTH']
        
        current_month_hurricane = hurricane_df[(hurricane_df['YEAR'] == current_year) & (hurricane_df['MONTH'] == current_month)]
        current_month_hurricane.reset_index(drop=True, inplace=True)

        for j in range(current_month_hurricane.shape[0]):
            if current_status == 2:
              break
            new_status = is_exposed_3(zillow_data_copy.loc[i, 'LAT'], zillow_data_copy.loc[i, 'LON'], 
                          current_month_hurricane.loc[j, 'LAT'], current_month_hurricane.loc[j, 'LON'], 
                          current_month_hurricane.loc[j, 'RADIUS_KM'])
            if (current_status == 0 and (new_status == 1 or new_status == 2)) or (current_status == 1 and new_status == 2):
                zillow_data_copy.loc[i, 'Exposed'] = new_status
                current_status = new_status   

    return zillow_data_copy

In [49]:
# filtered zillow data + 1 month hurricane
# Took 10 minutes to run
# updated_zillow_with_exposed_status_1_month = update_exposed_status(zillow_data_with_date, filtered_hurricane_df_1_month)
# updated_zillow_with_exposed_status_1_month

,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,LAT,LON,Date,HomeValue,Datetime,YEAR,MONTH,DAY,Exposed
0,25290,12,Jacksonville,city,FL,FL,"Jacksonville, FL",Duval County,30.3322,-81.6749,2000-01-31,102258.0,2000-01-31,2000,1,31,0
1,24043,13,Charlotte,city,NC,NC,"Charlotte-Concord-Gastonia, NC-SC",Mecklenburg County,35.2083,-80.8303,2000-01-31,156927.0,2000-01-31,2000,1,31,0
2,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.3370,2000-01-31,114429.0,2000-01-31,2000,1,31,0
3,12700,29,Miami,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Miami-Dade County,25.7840,-80.2101,2000-01-31,145831.0,2000-01-31,2000,1,31,0
4,54047,40,Raleigh,city,NC,NC,"Raleigh-Cary, NC",Wake County,35.8324,-78.6429,2000-01-31,176360.0,2000-01-31,2000,1,31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605539,6461,28439,Penney Farms,city,FL,FL,"Jacksonville, FL",Clay County,29.9807,-81.8108,2022-12-31,267397.0,2022-12-31,2022,12,31,0
605540,49256,28439,Dillsboro,city,NC,NC,"Cullowhee, NC",Jackson County,35.3706,-83.2527,2022-12-31,308937.0,2022-12-31,2022,12,31,0
605541,26569,28439,Polk City,city,FL,FL,"Lakeland-Winter Haven, FL",Polk County,28.1805,-81.8296,2022-12-31,249046.0,2022-12-31,2022,12,31,0
605542,42861,28439,Oak Park,city,GA,GA,NaN,Emanuel County,32.3822,-82.3153,2022-12-31,68641.0,2022-12-31,2022,12,31,0


In [ ]:
# filtered zillow data + 1 year hurricane

# updated_zillow_with_exposed_status_2022 = update_exposed_status(zillow_data_with_date, filtered_hurricane_df_2022)
# updated_zillow_with_exposed_status_2022

In [93]:
# filtered zillow data + 2 year hurricane
# Took 12 minutes to run

updated_zillow_with_exposed_status_2021_2022 = update_exposed_status(zillow_data_with_date, filtered_hurricane_df_2021_2022)
updated_zillow_with_exposed_status_2021_2022

,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,LAT,LON,Date,HomeValue,Datetime,YEAR,MONTH,DAY,Exposed
0,25290,12,Jacksonville,city,FL,FL,"Jacksonville, FL",Duval County,30.3322,-81.6749,2000-01-31,102258.0,2000-01-31,2000,1,31,0
1,24043,13,Charlotte,city,NC,NC,"Charlotte-Concord-Gastonia, NC-SC",Mecklenburg County,35.2083,-80.8303,2000-01-31,156927.0,2000-01-31,2000,1,31,0
2,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.3370,2000-01-31,114429.0,2000-01-31,2000,1,31,0
3,12700,29,Miami,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Miami-Dade County,25.7840,-80.2101,2000-01-31,145831.0,2000-01-31,2000,1,31,0
4,54047,40,Raleigh,city,NC,NC,"Raleigh-Cary, NC",Wake County,35.8324,-78.6429,2000-01-31,176360.0,2000-01-31,2000,1,31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490976,6461,28439,Penney Farms,city,FL,FL,"Jacksonville, FL",Clay County,29.9807,-81.8108,2022-12-31,267397.0,2022-12-31,2022,12,31,0
490977,49256,28439,Dillsboro,city,NC,NC,"Cullowhee, NC",Jackson County,35.3706,-83.2527,2022-12-31,308937.0,2022-12-31,2022,12,31,0
490978,26569,28439,Polk City,city,FL,FL,"Lakeland-Winter Haven, FL",Polk County,28.1805,-81.8296,2022-12-31,249046.0,2022-12-31,2022,12,31,0
490979,42861,28439,Oak Park,city,GA,GA,NaN,Emanuel County,32.3822,-82.3153,2022-12-31,68641.0,2022-12-31,2022,12,31,0


In [94]:
# Make a copy of zillow_data_with_exposed_status

zillow_data_with_date_copy = updated_zillow_with_exposed_status_2021_2022
zillow_data_with_date_copy

,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,LAT,LON,Date,HomeValue,Datetime,YEAR,MONTH,DAY,Exposed
0,25290,12,Jacksonville,city,FL,FL,"Jacksonville, FL",Duval County,30.3322,-81.6749,2000-01-31,102258.0,2000-01-31,2000,1,31,0
1,24043,13,Charlotte,city,NC,NC,"Charlotte-Concord-Gastonia, NC-SC",Mecklenburg County,35.2083,-80.8303,2000-01-31,156927.0,2000-01-31,2000,1,31,0
2,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.3370,2000-01-31,114429.0,2000-01-31,2000,1,31,0
3,12700,29,Miami,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Miami-Dade County,25.7840,-80.2101,2000-01-31,145831.0,2000-01-31,2000,1,31,0
4,54047,40,Raleigh,city,NC,NC,"Raleigh-Cary, NC",Wake County,35.8324,-78.6429,2000-01-31,176360.0,2000-01-31,2000,1,31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490976,6461,28439,Penney Farms,city,FL,FL,"Jacksonville, FL",Clay County,29.9807,-81.8108,2022-12-31,267397.0,2022-12-31,2022,12,31,0
490977,49256,28439,Dillsboro,city,NC,NC,"Cullowhee, NC",Jackson County,35.3706,-83.2527,2022-12-31,308937.0,2022-12-31,2022,12,31,0
490978,26569,28439,Polk City,city,FL,FL,"Lakeland-Winter Haven, FL",Polk County,28.1805,-81.8296,2022-12-31,249046.0,2022-12-31,2022,12,31,0
490979,42861,28439,Oak Park,city,GA,GA,NaN,Emanuel County,32.3822,-82.3153,2022-12-31,68641.0,2022-12-31,2022,12,31,0


In [95]:
zillow_data_with_date_copy.to_csv("zillow_with_exposed_2021_2022.csv", index=False)


In [53]:
# Testing
# exposed_data = zillow_data_with_date_copy[(zillow_data_with_date_copy['Exposed'] == 2) & (zillow_data_with_date_copy['State'] == 'FL') & (zillow_data_with_date_copy['CityName'].isin(['Cape Coral']))]
# exposed_data



,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,LAT,LON,Date,HomeValue,Datetime,YEAR,MONTH,DAY,Exposed
596800,30742,168,Cape Coral,city,FL,FL,"Cape Coral-Fort Myers, FL",Lee County,26.6443,-81.9957,2022-09-30,426538.0,2022-09-30,2022,9,30,2


In [96]:
exposed_cities = zillow_data_with_date_copy.loc[zillow_data_with_date_copy['Exposed'] == 2, 'CityName'].unique()
exposed_cities



array(['Baton Rouge', 'Houma', 'Denham Springs', 'Hammond',
       'Prairieville', 'Gonzales', 'Ponchatoula', 'Zachary', 'Baker',
       'Walker', 'Port Allen', 'Plaquemine', 'Livingston',
       'Donaldsonville', 'Tickfaw', 'Kentwood', 'Independence', 'Reserve',
       'Jackson', 'Lockport', 'Montegut', 'Clinton', 'Bourg', 'Paulina',
       'Sorrento', 'Roseland', 'Brusly', 'Lutcher', 'Albany', 'Chauvin',
       'Gramercy', 'Greensburg', 'Des Allemands', 'Garyville',
       'Slaughter', 'Paradis', 'French Settlement', 'Addis', 'Convent',
       'Dulac', 'Killian', 'Port Vincent', 'Cape Coral', 'Myrtle Beach',
       'Port Charlotte', 'Sumter', 'Conway', 'North Port', 'Florence',
       'Punta Gorda', 'North Fort Myers', 'Sebring', 'Lumberton',
       'Raeford', 'Arcadia', 'Hope Mills', 'Murrells Inlet', 'Hartsville',
       'Englewood', 'Avon Park', 'Rockingham', 'Darlington', 'Georgetown',
       'Lake Placid', 'Little River', 'Whiteville', 'Bennettsville',
       'Loris', 'Pawleys I

In [57]:
# city_list = ['Cape Coral', 'North Port']
# Sample_City_Testing = zillow_data_with_date_copy[(zillow_data_with_date_copy['CityName'].isin(city_list)) & (zillow_data_with_date_copy['Date'] >= '2022-09-01')]
# Sample_City_Testing

,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,LAT,LON,Date,HomeValue,Datetime,YEAR,MONTH,DAY,Exposed
596800,30742,168,Cape Coral,city,FL,FL,"Cape Coral-Fort Myers, FL",Lee County,26.6443,-81.9957,2022-09-30,426538.0,2022-09-30,2022,9,30,2
596918,13031,711,North Port,city,FL,FL,"North Port-Sarasota-Bradenton, FL",Sarasota County,27.0576,-82.1977,2022-09-30,391127.0,2022-09-30,2022,9,30,2
598994,30742,168,Cape Coral,city,FL,FL,"Cape Coral-Fort Myers, FL",Lee County,26.6443,-81.9957,2022-10-31,426076.0,2022-10-31,2022,10,31,0
599112,13031,711,North Port,city,FL,FL,"North Port-Sarasota-Bradenton, FL",Sarasota County,27.0576,-82.1977,2022-10-31,388993.0,2022-10-31,2022,10,31,0
601188,30742,168,Cape Coral,city,FL,FL,"Cape Coral-Fort Myers, FL",Lee County,26.6443,-81.9957,2022-11-30,423370.0,2022-11-30,2022,11,30,0
601306,13031,711,North Port,city,FL,FL,"North Port-Sarasota-Bradenton, FL",Sarasota County,27.0576,-82.1977,2022-11-30,389198.0,2022-11-30,2022,11,30,0
603382,30742,168,Cape Coral,city,FL,FL,"Cape Coral-Fort Myers, FL",Lee County,26.6443,-81.9957,2022-12-31,418922.0,2022-12-31,2022,12,31,0
603500,13031,711,North Port,city,FL,FL,"North Port-Sarasota-Bradenton, FL",Sarasota County,27.0576,-82.1977,2022-12-31,388849.0,2022-12-31,2022,12,31,0


In [97]:
# Orderby RegionID, YEAR, MONTH, and DAY
# Get the previous expose status, if exposed in the previous 6 months, set prev_exposed = 1 otherwise prev_expose = 0
# All southeastern states & 1 month hurricane -> Took 50 minutes to run 
# Filtered zillow & 2 year hurricane -> Tool 18 minutes to run

sorted_zillow_data_with_date_copy = zillow_data_with_date_copy.sort_values(['RegionID', 'YEAR', 'MONTH', 'DAY'], ascending=True)
sorted_zillow_data_with_date_copy.reset_index(drop=True, inplace=True)

def get_prev_expose_status(row, range):
    idx = row.name
    region_id = row['RegionID']
    region_id_rows = sorted_zillow_data_with_date_copy[sorted_zillow_data_with_date_copy['RegionID'] == region_id]
    region_id_idx = region_id_rows.index.get_loc(idx)
    if region_id_idx >= range:
        prev_rows = region_id_rows.iloc[region_id_idx-range:region_id_idx]
        expose_status_list = list(prev_rows['Exposed'])
        return expose_status_list
    else:
        return [0] * (range - region_id_idx) + list(region_id_rows.iloc[:region_id_idx]['Exposed'])

sorted_zillow_data_with_date_copy['Prev_Exposed_List_6M'] = sorted_zillow_data_with_date_copy.apply(lambda x: get_prev_expose_status(x, 6), axis=1)

sorted_zillow_data_with_date_copy['Prev_Exposed_6M'] = sorted_zillow_data_with_date_copy['Prev_Exposed_List_6M'].apply(lambda x: 1 if 2 in x else 0)

sorted_zillow_data_with_date_copy['Prev_Exposed_List_1Y'] = sorted_zillow_data_with_date_copy.apply(lambda x: get_prev_expose_status(x, 12), axis=1)

sorted_zillow_data_with_date_copy['Prev_Exposed_1Y'] = sorted_zillow_data_with_date_copy['Prev_Exposed_List_1Y'].apply(lambda x: 1 if 2 in x else 0)

sorted_zillow_data_with_date_copy

,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,LAT,LON,...,HomeValue,Datetime,YEAR,MONTH,DAY,Exposed,Prev_Exposed_List_6M,Prev_Exposed_6M,Prev_Exposed_List_1Y,Prev_Exposed_1Y
0,3301,4198,Aberdeen,city,NC,NC,"Pinehurst-Southern Pines, NC",Moore County,35.1350,-79.4326,...,171955.0,2008-03-31,2008,3,31,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
1,3301,4198,Aberdeen,city,NC,NC,"Pinehurst-Southern Pines, NC",Moore County,35.1350,-79.4326,...,171992.0,2008-04-30,2008,4,30,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
2,3301,4198,Aberdeen,city,NC,NC,"Pinehurst-Southern Pines, NC",Moore County,35.1350,-79.4326,...,172285.0,2008-05-31,2008,5,31,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
3,3301,4198,Aberdeen,city,NC,NC,"Pinehurst-Southern Pines, NC",Moore County,35.1350,-79.4326,...,172323.0,2008-06-30,2008,6,30,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
4,3301,4198,Aberdeen,city,NC,NC,"Pinehurst-Southern Pines, NC",Moore County,35.1350,-79.4326,...,172148.0,2008-07-31,2008,7,31,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490976,786869,13780,Westlake,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Palm Beach County,26.7549,-80.2984,...,679119.0,2022-08-31,2022,8,31,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
490977,786869,13780,Westlake,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Palm Beach County,26.7549,-80.2984,...,683809.0,2022-09-30,2022,9,30,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
490978,786869,13780,Westlake,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Palm Beach County,26.7549,-80.2984,...,683203.0,2022-10-31,2022,10,31,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
490979,786869,13780,Westlake,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Palm Beach County,26.7549,-80.2984,...,683100.0,2022-11-30,2022,11,30,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0


In [98]:
zillow_with_expose_and_pre_exp = sorted_zillow_data_with_date_copy
zillow_with_expose_and_pre_exp

,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,LAT,LON,...,HomeValue,Datetime,YEAR,MONTH,DAY,Exposed,Prev_Exposed_List_6M,Prev_Exposed_6M,Prev_Exposed_List_1Y,Prev_Exposed_1Y
0,3301,4198,Aberdeen,city,NC,NC,"Pinehurst-Southern Pines, NC",Moore County,35.1350,-79.4326,...,171955.0,2008-03-31,2008,3,31,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
1,3301,4198,Aberdeen,city,NC,NC,"Pinehurst-Southern Pines, NC",Moore County,35.1350,-79.4326,...,171992.0,2008-04-30,2008,4,30,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
2,3301,4198,Aberdeen,city,NC,NC,"Pinehurst-Southern Pines, NC",Moore County,35.1350,-79.4326,...,172285.0,2008-05-31,2008,5,31,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
3,3301,4198,Aberdeen,city,NC,NC,"Pinehurst-Southern Pines, NC",Moore County,35.1350,-79.4326,...,172323.0,2008-06-30,2008,6,30,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
4,3301,4198,Aberdeen,city,NC,NC,"Pinehurst-Southern Pines, NC",Moore County,35.1350,-79.4326,...,172148.0,2008-07-31,2008,7,31,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490976,786869,13780,Westlake,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Palm Beach County,26.7549,-80.2984,...,679119.0,2022-08-31,2022,8,31,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
490977,786869,13780,Westlake,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Palm Beach County,26.7549,-80.2984,...,683809.0,2022-09-30,2022,9,30,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
490978,786869,13780,Westlake,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Palm Beach County,26.7549,-80.2984,...,683203.0,2022-10-31,2022,10,31,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
490979,786869,13780,Westlake,city,FL,FL,"Miami-Fort Lauderdale-Pompano Beach, FL",Palm Beach County,26.7549,-80.2984,...,683100.0,2022-11-30,2022,11,30,0,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0


In [99]:
zillow_with_expose_and_pre_exp.to_csv("zillow_with_expose_and_pre_exp.csv", index=False)


In [100]:
exposed_cities = zillow_with_expose_and_pre_exp.loc[zillow_with_expose_and_pre_exp['Exposed'] == 2, 'CityName'].unique()
exposed_cities

array(['Atlantic Beach', 'Bishopville', 'Brusly', 'Dillon',
       'Donaldsonville', 'Englewood', 'Hartsville', 'Jackson',
       'Jamestown', 'Lake View', 'Lake Waccamaw', 'Little River',
       'Tabor City', 'Vero Beach', 'Zolfo Springs', 'Chadbourn',
       'Calabash', 'Lane', 'Latta', 'Manning', 'Oak Hill', 'Rowland',
       'Lutcher', 'Scranton', 'Coward', 'Dalzell', 'Denham Springs',
       'Fairmont', 'Hamlet', 'Holden Beach', 'Jensen Beach', 'Killian',
       'Lynchburg', 'North Port', 'Pine Ridge', 'Port Charlotte',
       'Prairieville', 'Rockingham', 'Rockledge', 'Sebastian',
       'Shallotte', 'Sunset Beach', 'Timmonsville', 'West Melbourne',
       'Green Sea', 'Orrum', 'Paulina', 'Livingston', 'Sanibel',
       'Mullins', 'Alcolu', 'Gonzales', 'Indiantown', 'Johnsonville',
       'Lake Placid', 'Lockport', 'Melbourne', 'Micco', 'Pamplico',
       'Satellite Beach', 'Sorrento', 'Wagram', 'Wallace', 'Red Springs',
       'Bokeelia', 'Cerro Gordo', 'Chauvin', 'Elizabethtown

In [109]:
#(zillow_with_expose_and_pre_exp['Date'] >= '2022-09-01')
city_list = ['Atlantic Beach', 'North Port']
#Sample_City_Testing = zillow_with_expose_and_pre_exp[(zillow_with_expose_and_pre_exp['CityName'].isin(city_list)) & (zillow_with_expose_and_pre_exp['Exposed'] == 2)]
Sample_City_Testing = zillow_with_expose_and_pre_exp[(zillow_with_expose_and_pre_exp['Exposed'] == 2) & (zillow_with_expose_and_pre_exp['Date'] <= '2022-09-01')]

Sample_City_Testing

,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,LAT,LON,...,HomeValue,Datetime,YEAR,MONTH,DAY,Exposed,Prev_Exposed_List_6M,Prev_Exposed_6M,Prev_Exposed_List_1Y,Prev_Exposed_1Y
5011,3861,10013,Brusly,city,LA,LA,"Baton Rouge, LA",West Baton Rouge Parish,30.3943,-91.2517,...,245492.0,2021-08-31,2021,8,31,2,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
11413,4387,4662,Donaldsonville,city,LA,LA,"Baton Rouge, LA",Ascension Parish,30.0954,-90.9925,...,170497.0,2021-08-31,2021,8,31,2,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
21320,5308,7149,Jackson,city,LA,LA,"Baton Rouge, LA",East Feliciana Parish,30.8346,-91.2080,...,129612.0,2021-08-31,2021,8,31,2,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
59056,9607,10087,Lutcher,city,LA,LA,"New Orleans-Metairie, LA",Saint James Parish,30.0646,-90.7124,...,137036.0,2021-08-31,2021,8,31,2,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
72522,11087,595,Denham Springs,city,LA,LA,"Baton Rouge, LA",Livingston Parish,30.4743,-90.9593,...,201683.0,2021-08-31,2021,8,31,2,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
83528,12197,24214,Killian,city,LA,LA,"Baton Rouge, LA",Livingston Parish,30.3530,-90.5795,...,233259.0,2021-08-31,2021,8,31,2,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
96779,13386,1248,Prairieville,city,LA,LA,"Baton Rouge, LA",Ascension Parish,30.3151,-90.9571,...,272589.0,2021-08-31,2021,8,31,2,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
112330,15657,9084,Paulina,city,LA,LA,"New Orleans-Metairie, LA",Saint James Parish,30.0351,-90.7226,...,246451.0,2021-08-31,2021,8,31,2,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
113191,15831,4253,Livingston,city,LA,LA,"Baton Rouge, LA",Livingston Parish,30.4953,-90.7467,...,195749.0,2021-08-31,2021,8,31,2,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
130803,18320,1331,Gonzales,city,LA,LA,"Baton Rouge, LA",Ascension Parish,30.2132,-90.9234,...,231164.0,2021-08-31,2021,8,31,2,"[0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0


In [ ]:
# orlando_data = zillow_with_expose_and_pre_exp[(zillow_with_expose_and_pre_exp['CityName'].isin(['Orlando']))]
# orlando_data

,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,LAT,LON,Date,HomeValue,Datetime,YEAR,MONTH,DAY,expose_status,prev_expose_status_list,prev_exposed
205896,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.337,2000-01-31,114429.0,2000-01-31,2000,1,31,Safe,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
205897,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.337,2000-02-29,114802.0,2000-02-29,2000,2,29,Safe,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
205898,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.337,2000-03-31,115288.0,2000-03-31,2000,3,31,Safe,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
205899,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.337,2000-04-30,116072.0,2000-04-30,2000,4,30,Safe,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
205900,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.337,2000-05-31,116811.0,2000-05-31,2000,5,31,Safe,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206167,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.337,2022-08-31,384365.0,2022-08-31,2022,8,31,Safe,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
206168,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.337,2022-09-30,386208.0,2022-09-30,2022,9,30,Exposed,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
206169,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.337,2022-10-31,387141.0,2022-10-31,2022,10,31,Safe,"[Safe, Safe, Safe, Safe, Safe, Exposed]",1
206170,13121,20,Orlando,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4773,-81.337,2022-11-30,387136.0,2022-11-30,2022,11,30,Safe,"[Safe, Safe, Safe, Safe, Exposed, Safe]",1


In [ ]:
# zillow_exposed_city_in_Sep2022 = zillow_with_expose_and_pre_exp[(zillow_with_expose_and_pre_exp['CityName'].isin(exposed_cities)) & ((zillow_with_expose_and_pre_exp['expose_status'] == 'Exposed'))]

# zillow_exposed_city_in_Sep2022

,RegionID,SizeRank,CityName,RegionType,StateName,State,Metro,CountyName,LAT,LON,Date,HomeValue,Datetime,YEAR,MONTH,DAY,expose_status,prev_expose_status_list,prev_exposed
3032,3476,26422,Atlantic Beach,city,SC,SC,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",Horry County,33.8035,-78.7177,2022-09-30,324907.0,2022-09-30,2022,9,30,Exposed,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
8828,3681,4938,Bishopville,city,SC,SC,NaN,Lee County,34.2202,-80.2483,2022-09-30,121822.0,2022-09-30,2022,9,30,Exposed,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
26768,4370,3826,Dillon,city,SC,SC,NaN,Dillon County,34.4233,-79.3689,2022-09-30,135283.0,2022-09-30,2022,9,30,Exposed,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
28700,4593,1985,Englewood,city,FL,FL,"North Port-Sarasota-Bradenton, FL",Sarasota County,26.9717,-82.3524,2022-09-30,441035.0,2022-09-30,2022,9,30,Exposed,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
40292,5082,1935,Hartsville,city,SC,SC,"Florence, SC",Darlington County,34.3675,-80.0829,2022-09-30,143310.0,2022-09-30,2022,9,30,Exposed,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091852,220294,25258,Briarcliffe Acres,city,SC,SC,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",Horry County,33.7901,-78.7490,2022-09-30,893855.0,2022-09-30,2022,9,30,Exposed,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
1111724,396841,10268,Holden Heights,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.5013,-81.3878,2022-09-30,372367.0,2022-09-30,2022,9,30,Exposed,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
1112000,396842,4202,Conway,city,FL,FL,"Orlando-Kissimmee-Sanford, FL",Orange County,28.4968,-81.3316,2022-09-30,432492.0,2022-09-30,2022,9,30,Exposed,"[Safe, Safe, Safe, Safe, Safe, Safe]",0
1114202,396878,12837,Midway,city,FL,FL,"Tallahassee, FL",Gadsden County,28.7924,-81.2314,2022-09-30,258080.0,2022-09-30,2022,9,30,Exposed,"[Safe, Safe, Safe, Safe, Safe, Safe]",0


In [ ]:
# mean_lat = zillow_exposed_city_in_Sep2022["LAT"].mean()
# mean_lon = zillow_exposed_city_in_Sep2022["LON"].mean()

# map = folium.Map(location=[mean_lat, mean_lon], zoom_start=10)

# for i, city in zillow_exposed_city_in_Sep2022.iterrows():
#     folium.CircleMarker(
#         location=[city['LAT'], city['LON']],
#         radius=3,
#         color='red',
#         fill=True,
#         fill_color='red',
#         tooltip=city['CityName']
#     ).add_to(map)

# map